In [1]:
from sagas.modules.life.data_source import engine, restaurant, hotel, metadata
import pandas as pd
import json

# table_name='hotel'
table_name='restaurant'
query = f"select * from {table_name}"
df = pd.read_sql_query(query, engine)
df

,id,name,cuisine,price_range,outside_seating
0,1,Donath,Italian,mid-range,1
1,2,Berlin Burrito Company,Mexican,cheap,0
2,3,I due forni,Italian,mid-range,1
3,4,Lụa Restaurant,Vietnamese,cheap,1
4,5,Pfefferberg,German,mid-range,1
5,6,Marubi Ramen,Japanese,cheap,0
6,7,Gong Gan,Korean,cheap,1


In [2]:
# restaurant
[t.name for t in metadata.sorted_tables]

['hotel', 'restaurant', 'users', 'addresses']

In [3]:
for primary_key in hotel.primary_key:
    print('pk', primary_key)
for c in hotel.c:
    print(c.name, c.type)

pk hotel.id
id INTEGER
name VARCHAR(100)
city VARCHAR(50)
price_range VARCHAR(50)
breakfast_included BOOLEAN
free_wifi BOOLEAN
swimming_pool BOOLEAN
star_rating INTEGER


In [23]:
from typing import Text, Any, Dict, List, Union, Optional, Tuple, Set
from dataclasses import dataclass
from sagas.nlu.anal import build_anal_tree, Doc, AnalNode
from sagas.nlu.anal_corpus import model_info
from sagas.modules.life.data_source import engine, restaurant, hotel, metadata
from sagas.nlu.anal_expr import match
from sagas.nlu.anal_data_types import behave_, desc_, phrase_, rel_, path_, _
import pandas as pd
import json
from pprint import pprint
    
class agent_base(object):
    def __init__(self, ds):
        self.ds=ds
        
@dataclass
class ds_meta:
    agent: Any
    cond: List[Any]

class restaurant_agent(agent_base):
    def browse(self, pred):
        table_name=self.ds.name
        query = f"select * from {table_name} limit 2"
        df = pd.read_sql_query(query, engine)
        return json.loads(df.to_json(orient='records'))
    def __call__(self, f):
        r=match(f,
            behave_(_, 'quote|引用', _, _), self.browse,
            behave_(_, 'perception|感知', _, _), lambda arg: 'perception',
            behave_(_, 'perception|感知', desc_('result|结果', _), _), lambda arg: arg.text,
            _, None
            )
        return r
    
class hotel_agent(agent_base):
    def __call__(self, f):
        print(f.text)
        return f.text
    
meta_ls=[ds_meta(restaurant_agent(restaurant), 
                 ['InstitutePlace|场所', 
                  'location: eat|吃',
                  'domain: commerce|商业'
                 ]),
         ds_meta(hotel_agent(hotel), 
                 ['InstitutePlace|场所', 
                 'location: eat|吃',
                 'location: reside|住下',
                 'domain: commerce|商业'
                 ]),
        ]

f=build_anal_tree('list some restaurants', 'en', 'stanza')
f.draw()
target=f.model().target
print(target.text, target.types, 
      target.match('InstitutePlace|场所'),
      target.match('location: reside|住下; location: eat|吃'),
      target.match('+location: reside|住下; location: eat|吃'),
     )

for m in meta_ls:
    print('-'*25)
    
    rset=[(cond, target.match(cond)) for cond in m.cond]
    succ=all([r for c,r in rset])
    print('✔' if succ else '✘', m.agent.ds.name, rset)
    if succ:
        r=m.agent(f)    
        if r:
            pprint(r)
        else:
            print('result -> _')

root: list (list, verb)
|-- obj: restaurants (restaurant, noun)
|   +-- det: some (some, det)
+-- punct: . (., punct)
restaurants {'InstitutePlace|场所'} True True False
-------------------------
✔ restaurant [('InstitutePlace|场所', True), ('location: eat|吃', True), ('domain: commerce|商业', True)]
[{'cuisine': 'Italian',
  'id': 1,
  'name': 'Donath',
  'outside_seating': 1,
  'price_range': 'mid-range'},
 {'cuisine': 'Mexican',
  'id': 2,
  'name': 'Berlin Burrito Company',
  'outside_seating': 0,
  'price_range': 'cheap'}]
-------------------------
✘ hotel [('InstitutePlace|场所', True), ('location: eat|吃', True), ('location: reside|住下', False), ('domain: commerce|商业', True)]


In [17]:
from sagas.nlu.anal import build_anal_tree, Doc, AnalNode
from sagas.nlu.anal_corpus import model_info
sents=['list some restaurants',
      'what restaurants can you recommend?',
       'what is the price range of Berlin Burrito Company',
      ]
for s in sents:
    f=build_anal_tree(s, 'en', 'stanza')
    print(f.doc.sents)
    f.draw()
    print(f.model().target.text, f.model().target.types)

list some restaurants
root: list (list, verb)
|-- obj: restaurants (restaurant, noun)
|   +-- det: some (some, det)
+-- punct: . (., punct)
restaurants {'InstitutePlace|场所'}
what restaurants can you recommend?
root: recommend (recommend, verb)
|-- obj: restaurants (restaurant, noun)
|   +-- det: what (what, det)
|-- aux: can (can, aux)
|-- nsubj: you (you, pron)
+-- punct: ? (?, punct)
restaurants {'InstitutePlace|场所'}
what is the price range of Berlin Burrito Company
root: what (what, pron)
|-- cop: is (be, aux)
|-- nsubj: range (range, noun)
|   |-- det: the (the, det)
|   |-- compound: price (price, noun)
|   +-- nmod: Company (Company, propn)
|       |-- case: of (of, adp)
|       |-- compound: Berlin (Berlin, propn)
|       +-- compound: Burrito (Burrito, propn)
+-- punct: . (., punct)
range {'facilities|设施', 'Range|幅度', 'Distance|距离'}


In [16]:
from sagas.nlu.anal import build_anal_tree, Doc, AnalNode
from sagas.nlu.anal_corpus import model_info
f=build_anal_tree('list some restaurants', 'en', 'stanza')
f.draw()
print(f.model().target.text, f.model().target.types)

root: list (list, verb)
|-- obj: restaurants (restaurant, noun)
|   +-- det: some (some, det)
+-- punct: . (., punct)
restaurants {'InstitutePlace|场所'}


In [12]:
from sagas.nlu.anal import build_anal_tree, Doc, AnalNode
from sagas.nlu.anal_corpus import model_info
sents=['附近有什么好吃的饭馆',
       '你推荐我吃什么菜',
       '最便宜的旅馆是哪家？',
      ]
for i,s in enumerate(sents):
    print(i, s)
    for engine in ('stanza', 'ltp'):
        f=build_anal_tree(s, 'zh', engine)
        f.draw()
        if f.model().target:
            print(f.model().target.text, f.model().target.types)
        print('*** '*8)

0 附近有什么好吃的饭馆
root: 有 (有, verb)
|-- nsubj: 附近 (附近, noun)
+-- obj: 饭馆 (饭馆, noun)
    |-- det: 什么 (什么, pron)
    +-- amod: 好吃 (好吃, adj)
        +-- mark:relcl: 的 (的, part)
饭馆 {'InstitutePlace|场所'}
*** *** *** *** *** *** *** *** 
root: 有 (有, verb)
|-- adv: 附近 (附近, noun)
+-- obj: 饭馆 (饭馆, noun)
    |-- att: 什么 (什么, pron)
    +-- att: 好吃 (好吃, adj)
        +-- rad: 的 (的, part)
饭馆 {'InstitutePlace|场所'}
*** *** *** *** *** *** *** *** 
1 你推荐我吃什么菜
root: 吃 (吃, verb)
|-- nsubj: 你 (你, pron)
|-- advcl: 推荐 (推荐, verb)
|   +-- obj: 我 (我, pron)
+-- obj: 菜 (菜, noun)
    +-- det: 什么 (什么, pron)
菜 {'part|部件', 'vegetable|蔬菜', 'food|食品'}
*** *** *** *** *** *** *** *** 
root: 推荐 (推荐, verb)
|-- nsubj: 你 (你, pron)
|-- dbl: 我 (我, pron)
+-- obj: 吃 (吃, verb)
    +-- obj: 菜 (菜, noun)
        +-- att: 什么 (什么, pron)
吃 {'eat|吃', 'destroy|消灭', 'exhaust|损耗', 'absorb|吸收', 'alive|活着', 'suffer|遭受'}
*** *** *** *** *** *** *** *** 
2 最便宜的旅馆是哪家？
root: 哪家 (哪家, adj)
|-- nsubj: 旅馆 (旅馆, noun)
|   +-- amod: 便宜 (便宜, adj)
|       |

In [8]:
from sqlalchemy import Table, Column, Integer, String, Boolean, MetaData, ForeignKey, create_engine
engine = create_engine('sqlite:///:memory:')
metadata = MetaData()

user = Table('user', metadata,
    Column('user_id', Integer, primary_key=True),
    Column('user_name', String(16), nullable=False),
    Column('email_address', String(60), key='email'),
    Column('nickname', String(50), nullable=False)
)

user_prefs = Table('user_prefs', metadata,
    Column('pref_id', Integer, primary_key=True),
    Column('user_id', Integer, ForeignKey("user.user_id"), nullable=False),
    Column('pref_name', String(40), nullable=False),
    Column('pref_value', String(100))
)

metadata.create_all(engine)

In [11]:
[t.name for t in metadata.sorted_tables]

['employees', 'user', 'user_prefs']

In [10]:
employees = Table('employees', metadata,
    Column('employee_id', Integer, primary_key=True),
    Column('employee_name', String(60), nullable=False, key='name'),
    # Column('employee_dept', Integer, ForeignKey("departments.department_id"))
)
employees.create(engine)